# 🔮 Time-Series Forecasting
## Prophet & LSTM for Next-Day Price Prediction

In [ ]:
!pip install pandas matplotlib prophet tensorflow scikit-learn


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('../data/labeled_events_enhanced.csv', parse_dates=['date'])
btc = df[df['token'] == 'BTC'].sort_values('date')[['date', 'close']].rename(columns={'date': 'ds', 'close': 'y'})
btc.head()


In [ ]:
from prophet import Prophet

prophet = Prophet()
prophet.fit(btc)

future = prophet.make_future_dataframe(periods=7)
forecast = prophet.predict(future)

fig1 = prophet.plot(forecast)
plt.title("Bitcoin Price Forecast (Prophet)")
plt.show()


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Normalize
scaler = MinMaxScaler()
scaled = scaler.fit_transform(btc['y'].values.reshape(-1, 1))

# Prepare sequences
window_size = 5
X, y = [], []
for i in range(len(scaled) - window_size):
    X.append(scaled[i:i+window_size])
    y.append(scaled[i+window_size])
X, y = np.array(X), np.array(y)

# Train/test split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=20, verbose=0)
preds = model.predict(X_test)

# Plot
plt.plot(scaler.inverse_transform(y_test.reshape(-1,1)), label='Actual')
plt.plot(scaler.inverse_transform(preds), label='LSTM Forecast')
plt.legend()
plt.title("Bitcoin Price Forecast (LSTM)")
plt.show()
